In [1]:
import os
from data_ml_utils.pyathena_client.client import PyAthenaClient

os.environ["AWS_ACCESS_KEY_ID"] = "xxx"
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxx"

### initialise pyathena client

In [2]:
pyathena_client = PyAthenaClient()

### query with `pyathena_client`
- the way we are querying with athena, saves `int64` as pandas `Int64`
    - this data type is incompatible with most of the pandas function we will use
    - we will have to convert them to 
- almost one-liner to query results and return as pandas dataframe

In [3]:
query = """
SELECT
    *
FROM
    reporting_metrics_dev.int_active_tradies_count__tradie
LIMIT 10
"""

df_raw = pyathena_client.query_as_pandas(final_query=query)
df_raw.dtypes

num_active_tradies       int64
num_inactive_tradies     int64
date_created            object
dtype: object

In [4]:
df_raw.head()

,num_active_tradies,num_inactive_tradies,date_created
0,21159,316175,2022-03-02
1,21085,316394,2022-03-04
2,21055,316195,2022-03-01
3,21042,316439,2022-03-05


### drop athena table
- we will use `reporting_metrics_dev.int_active_tradies_count__tradie` as an example
- one-liner to drop athena table
- if successful returns 0, else 1

In [6]:
pyathena_client.drop_table(
    table_name="int_active_tradies_count__tradie",
    database="reporting_metrics_dev"
)

0

### creating table with schema (or updating partitions of table) in offline feature store
- if there are changes to schema, table has to be dropped and recreated again
- we will use `reporting_metrics_dev.int_active_tradies_count__tradie` as an example
- if successful returns 0, else 1

- one-liner to create table and update partitions

In [7]:
pyathena_client.create_msck_repair_table(
    create_raw_query="tutorial_sql_schema/create_table_schema.sql",
    repair_raw_query="tutorial_sql_schema/msck_repair_table.sql",
    yaml_schema_file_path="tutorial_sql_schema/int_active_tradies_count__tradie.yaml"
)

0